In [1]:
from numpy import array, sqrt, sin, cos, arctan2, set_printoptions

# Representation of float numbers in arrays when printed
set_printoptions(formatter={'float_kind':"{:.3f}".format})


# IS-GPS-200K constants
pi = 3.1415926535898         # Constant pi
c = 2.99792458e8             # Speed of light [meter/second]
OMEGADOTe = 7.2921151467e-5  # Earth's rotation rate [rad/s]
GM = 3.986005e14             # Earth's gravitational constant [meter^3/second^2]


# Correction for beginning or end of week crossovers in GNSS systems
def dt(t, t0):
    t = t - t0
    if t > 302400:
        t = t - 604800
    elif t < -302400:
        t = t + 604800
    return t


# Satellite ECEF position
def satpos(ttr, toe, ROOTa, DELTAn, M0, e, omega, Cus, Cuc, Crs, Crc, Cis, Cic, i0, iDOT, OMEGA0, OMEGADOT):
    
    # Anomalies of the Keplerian orbit
    a = ROOTa**2        # Semi-major axis [m]
    n0 = sqrt(GM/a**3)  # Mean angular velocity [rad/sec]
    t = dt(ttr, toe)    # Time from reference epoch [s]
    n = n0 + DELTAn     # Corrected mean motion [rad/s]
    M = M0 + n*t        # Mean anomaly [rad]

    # Kepler's equation
    epsilon = 1e-10
    E_new = M
    E = 0

    while abs(E_new - E) > epsilon:
        E = E_new
        E_new = M + e*sin(E)

    # Eccentric anomaly
    E = E_new

    # True anomaly
    v = arctan2(sqrt(1 - e**2)*sin(E), cos(E) - e)

    # Argument of latitude
    PHI = v + omega

    # Second harmonic perturbations
    du = Cus*sin(2*PHI) + Cuc*cos(2*PHI)  # Argument of latitude correction [rad]
    dr = Crs*sin(2*PHI) + Crc*cos(2*PHI)  # Radius correction [m]
    di = Cis*sin(2*PHI) + Cic*cos(2*PHI)  # Inclination correction[rad]

    # Orbit corrections
    u = PHI + du               # Corrected argument of latitude [rad]
    r = a*(1 - e*cos(E)) + dr  # Corrected radius [m]
    i = i0 + di + iDOT*t       # Corrected inclination [rad]

    # Corrected longitude of ascending node
    OMEGA = OMEGA0 + (OMEGADOT - OMEGADOTe)*t - OMEGADOTe*toe

    # Satellite position in ECEF system
    Xs0 = array([[r*cos(u)*cos(OMEGA) - r*sin(u)*sin(OMEGA)*cos(i)],
                 [r*cos(u)*sin(OMEGA) + r*sin(u)*cos(OMEGA)*cos(i)],
                 [r*sin(u)*sin(i)]])
    return Xs0


# z-axis rotation
def Rz(rz):
    return array([[cos(rz), -sin(rz), 0],
                  [sin(rz), cos(rz), 0],
                  [0, 0, 1]])

In [2]:
from georinex import load
from numpy.linalg import norm


# RINEX navigation file
navfile = "TRDS00NOR_S_20230390000_01H_GN.rnx"

# Read navigation file
nav = load(navfile)

# Extract data for satellite G32
prn32 = nav.sel(sv='G32')

# Approximate receiver position [m]
Xr = array([[2820171.1097],
            [513485.9012],
            [5678935.7475]])

# Satelllite G32 broadcast ephemerides (RINEX)
ttr = 8134                              # [s] @02:15:34
toe = 7200                              # [s] @02:00:00
sqrtA = prn32['sqrtA'].values[0]        # [sqrt(m)]
DeltaN = prn32['DeltaN'].values[0]      # [rad/s]
M0 = prn32['M0'].values[0]              # [rad]
ecc = prn32['Eccentricity'].values[0]   # [unitless]
omega = prn32['omega'].values[0]        # [rad]
Cus = prn32['Cus'].values[0]            # [rad]
Cuc = prn32['Cuc'].values[0]            # [rad]
Crs = prn32['Crs'].values[0]            # [m]
Crc = prn32['Crc'].values[0]            # [m]
Cis = prn32['Cis'].values[0]            # [rad]
Cic = prn32['Cic'].values[0]            # [rad]
i0 = prn32['Io'].values[0]              # [rad]
iDOT = prn32['IDOT'].values[0]          # [rad/s]
OMEGA0 = prn32['Omega0'].values[0]      # [rad]
OMEGADOT = prn32['OmegaDot'].values[0]  # [rad/s]

In [3]:
# Satellite ECEF position @02:15:34 [m]
Xs0 = satpos(ttr, toe, sqrtA, DeltaN, M0, ecc, omega,
             Cus, Cuc, Crs, Crc, Cis, Cic, i0, iDOT, OMEGA0, OMEGADOT)
print(Xs0)

[[-21686255.021]
 [14951498.205]
 [2568092.207]]


In [4]:
# Altitude above the Earths surface
R = 6371e3
print("Altitude above Earth: {:.3f} m".format(norm(Xs0) - R))

Altitude above Earth: 20094752.455 m


In [5]:
# Satellite position @ttr + 1s
Xs0_1 = satpos(ttr + 1, toe, sqrtA, DeltaN, M0, ecc, omega,
               Cus, Cuc, Crs, Crc, Cis, Cic, i0, iDOT, OMEGA0, OMEGADOT)

# Velocity in orbit [m/s]
print("Velocity in orbit: {:.3f} m/s".format(norm(Xs0_1 - Xs0)))

Velocity in orbit: 3193.135 m/s


In [6]:
# Initial estimate of signal delay [ms]
sd_new = norm(Xs0 - Xr)/c
sd = 0

# Estimate signal travel time due to earth rotation
Xs = None
while abs(sd_new - sd) > 1e-10:
    sd = sd_new
    Xs = Rz(-OMEGADOTe*sd)@Xs0

    # Compute delay estimate
    sd_new = norm(Xs - Xr)/c

In [7]:
# Corrected satellite ECEF position @02:15:34 [m]
print(Xs)

[[-21686150.961]
 [14951649.136]
 [2568092.207]]


In [8]:
# Estimate of signal delay [ms]
print("Signal delay: {:.3f} ms".format(norm(Xs - Xr)/c*1e3))

Signal delay: 95.442 ms


In [9]:
# Change in satellite position due to Earth rotation [m]
print("Change in satellite position: {:.3f} m".format(norm(Xs - Xs0)))

Change in satellite position: 183.326 m
